In [1]:
import csv
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Input, Dense, LSTM, Dropout
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint
from random import randint
import argparse

Using TensorFlow backend.


In [2]:
data = pd.read_excel('final_version.xlsx', header=None)
data = data.to_numpy()[1:]

In [3]:
idx = 0

In [4]:
X, Y = [], []

In [5]:
while True:
    if idx+15 >= len(data):
        break
    X.append(data[idx:idx+15])
    Y.append(data[idx+15])
    idx += 1
X = np.array(X)
Y = np.array(Y)

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)

In [7]:
X_train = np.array(X_train)[:, :, [1, 3, 4]]
Y_train = np.array(Y_train)[:, -1].reshape(-1, 1)
X_test = np.array(X_test)[:, :, [1, 3, 4]]
Y_test = np.array(Y_test)[:, -1].reshape(-1, 1)

In [8]:
np.shape(X_train)

(17527, 15, 3)

In [9]:
X_nonzero = []
Y_nonzero = []
for i, y in enumerate(Y_train):
    if y[-1] != 0.0:
        X_nonzero.append(X_train[i])
        Y_nonzero.append(y)
        
        

In [10]:
X_train = np.concatenate((X_train, X_nonzero, X_nonzero, X_nonzero))
Y_train = np.concatenate((Y_train, Y_nonzero, Y_nonzero, Y_nonzero))



In [16]:
def build_model(input_shape):
    x = Input(shape=input_shape)

    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(x)
    # h = LSTM(20, return_sequences=True, activation='tanh', kernel_initializer='he_normal')(h)
    # h = LSTM(20, activation='tanh', kernel_initializer='he_normal')(h)
    h = LSTM(80, activation='tanh',
             kernel_initializer='he_normal')(x)  # 128

    y = Dense(1, activation='relu')(h)
    return Model(inputs=x, outputs=y)

In [17]:
model = build_model((15, 3))
model.summary()


Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 15, 3)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 80)                26880     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 81        
Total params: 26,961
Trainable params: 26,961
Non-trainable params: 0
_________________________________________________________________


In [29]:
model.load_weights('./best_model_3000_80_Adam'+'.h5')

In [20]:
import matplotlib.pyplot as plt

In [30]:
plt.plot(hist.history['loss'], 'y', label='train_loss')
plt.plot(hist.history['val_loss'], 'r', label='val_loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend(loc='upper left')
plt.show()

NameError: name 'hist' is not defined

In [27]:
hist = model.fit(
    X_train, Y_train,
    batch_size=128,
    validation_split=0.2,
    epochs=20000,
    verbose=2,
    callbacks=[es, mc])

Train on 26768 samples, validate on 6692 samples
Epoch 1/20000


KeyboardInterrupt: 

In [24]:
es = EarlyStopping(monitor='val_mae', mode='min', verbose=1, patience=20)
mc = ModelCheckpoint('best_model.h5', monitor='val_mae', mode='min', save_best_only=True)

In [26]:
model.compile(loss='mse', optimizer=Adam(), metrics=['mae'])

In [32]:
loss, mae = model.evaluate(X_test, Y_test)

4382/4382 [==============================] - 0s 108us/step
